In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import io
from glob import glob
from tqdm import tqdm

In [ ]:
image_path='dataset/images/'
mask_path='dataset/masks/'

In [ ]:
train_images=[]
def load_images(path):
    images=glob(os.path.join(path,'*.jpg'))
    for i in tqdm(images):
        i=cv2.imread(i)
        i=cv2.resize(i, (128,128))
        i=cv2.normalize(i,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        train_images.append(i)
    return train_images

train_images=load_images(image_path)

In [ ]:
train_masks=[]
def load_images(path):
    images=glob(os.path.join(path,'*.png'))
    for i in tqdm(images):
        i=cv2.imread(i)
        i=cv2.resize(i, (128,128))
        i=cv2.normalize(i,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        train_masks.append(i)
    return train_masks

train_masks=load_images(mask_path)

In [ ]:
plt.imshow(train_images[0])

In [ ]:
plt.imshow(train_masks[0])

In [ ]:
train_images = np.array(train_images)
train_masks = np.array(train_masks)

In [ ]:
from sklearn.model_selection import train_test_split

trainx, testx, trainy, testy = train_test_split(train_images,train_masks, test_size=0.2, random_state=40)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

def U_net():
        # Build U-Net model
        inputs = Input((128, 128, 3))
        s = Lambda(lambda x: x / 255) (inputs)

        c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
        c1 = Dropout(0.1) (c1)
        c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
        p1 = MaxPooling2D((2, 2)) (c1)

        c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
        c2 = Dropout(0.1) (c2)
        c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
        p2 = MaxPooling2D((2, 2)) (c2)

        c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
        c3 = Dropout(0.2) (c3)
        c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
        p3 = MaxPooling2D((2, 2)) (c3)

        c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
        c4 = Dropout(0.2) (c4)
        c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
        p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

        c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
        c5 = Dropout(0.3) (c5)
        c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

        u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
        u6 = concatenate([u6, c4])
        c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
        c6 = Dropout(0.2) (c6)
        c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

        u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
        u7 = concatenate([u7, c3])
        c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
        c7 = Dropout(0.2) (c7)
        c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

        u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
        u8 = concatenate([u8, c2])
        c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
        c8 = Dropout(0.1) (c8)
        c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

        u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
        u9 = concatenate([u9, c1], axis=3)
        c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
        c9 = Dropout(0.1) (c9)
        c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

        outputs = Conv2D(3, (1, 1), activation='softmax') (c9)

        model = Model(inputs=[inputs], outputs=[outputs])
        return model

In [ ]:
model = U_net()
model.summary()

In [ ]:
plt.imshow(testy[25])

In [ ]:
plt.imshow(testx[25])

In [ ]:
def dice_coefficient(y_true,y_pred):
    numerator = 2*tf.reduce_sum(y_true*y_pred)
    denominator = tf.reduce_sum(y_true+y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

In [ ]:
def loss(y_true,y_pred):
    return binary_crossentropy(y_true,y_pred) - tf.math.log(dice_coefficient(y_true,y_pred) + tf.keras.backend.epsilon())

In [ ]:
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
model.compile('Adam', loss = loss, metrics=['accuracy'])

In [ ]:
model.fit(trainx, trainy, batch_size=8, epochs = 100, validation_data = (testx, testy))

In [ ]:
out = model.predict(np.expand_dims(testx[10],axis=0))[0]
plt.imshow((out * 255).astype(np.uint8))